In [1]:
!python -m pip install fuzzywuzzy

In [4]:
!python -m pip install python-levenshtein

In [11]:
!python -m pip install jaro-winkler

#### start working with the full dataset in each sheet

In [115]:
#define the SheetName from screen input, also creates the separate folder for the final data
import os
SheetName = input("please enter the name of the Sheet. This name is also used to create a new folder to same final data.")
current_directory = os.getcwd()
new_folder = os.path.join(current_directory,'data',SheetName)
os.makedirs(new_folder,exist_ok=True)

please enter the name of the Sheet. This name is also used to create a new folder to same final data.ICD10-nDOIDMedD


In [116]:
#test to use excel sheet as input
#separate the excelsheet on each EFO ID and names
#generate the scores
#output to another excelsheet

import pandas as pd
import jaro

file_path = "data/mapping_fuzz_match.xlsx"

df_a = pd.read_excel(file_path, sheet_name=SheetName) #sheetname needs to be keyboard input
 
print(df_a.head())

    METASTORE_ID                                 METASTORE_SYNONYMS  \
0  NVMDIS1091335  seborrhoeic wart|Basal Cell Papilloma|seborrhe...   
1    NVMDIS20382                     albinismus|Albinism|Albinismus   
2  NVMDIS1095484  Anaemia spherocytic|anemia spherocytic|Heredit...   
3    NVMDIS21274  Fragilitas Ossium|brittle bones|Brittle bones|...   
4  NVMDIS1070743  Sleep Apnea Syndrome|SLEEP APNOEA|apnea syndro...   

                                  EFO_IDS_FROM_ICD10  \
0                            EFO_0005584|EFO_1000686   
1  MONDO_0018306|MONDO_0019312|MONDO_0012220|MOND...   
2                        MONDO_0019050|MONDO_0019350   
3  MONDO_0009803|MONDO_0011215|MONDO_0017791|MOND...   
4            MONDO_0800026|EFO_0003877|MONDO_0020493   

                                EFO_NAMES_FROM_ICD10  
0     seborrheic keratosis|dermatosis papulosa nigra  
1  Griscelli syndrome|Hermansky-Pudlak syndrome|G...  
2  inherited hemoglobinopathy|hereditary spherocy...  
3  congenital os

In [117]:
#get the column names
EFO_column_names = df_a.columns[-2:].tolist()
print(EFO_column_names)
col1 = EFO_column_names[0]
col2 = EFO_column_names[1]


['EFO_IDS_FROM_ICD10', 'EFO_NAMES_FROM_ICD10']


In [118]:
def tranform_table(df_a,col1,col2):
    # Separate rows where 'METASTORE_SYNONYMS' is empty
    df_empty_syn = df_a[df_a['METASTORE_SYNONYMS'].isna() | df_a['METASTORE_SYNONYMS'].str.strip().eq('')]
    df_empty_syn = df_empty_syn.assign(EFO_ID_split='', EFO_Name_split='')

    # Apply the transformation to rows where 'METASTORE_SYNONYMS' is not empty
    df_b_non_empty_syn = df_a[df_a['METASTORE_SYNONYMS'].notna() & df_a['METASTORE_SYNONYMS'].str.strip().astype(bool)]
    df_b_non_empty_syn = df_b_non_empty_syn.assign(EFO_ID_split=df_b_non_empty_syn[col1].str.split('|'),EFO_Name_split=df_b_non_empty_syn[col2].str.split('|'))
    #Explode the list into separate row
    df_b_non_empty_syn=df_b_non_empty_syn.explode(['EFO_ID_split', 'EFO_Name_split']).reset_index(drop=True)
    
    # Combine the DataFrames
    #df_b = pd.concat([df_b_non_empty_syn, df_empty_syn]).reset_index(drop=True)

    return df_b_non_empty_syn, df_empty_syn

df_b,df_b_empty = tranform_table(df_a,col1,col2)

def filepath(file_name):
    return os.path.join(new_folder,file_name)

df_b.to_csv(filepath('mapping_fuzz_match_3.csv'),index=False)

df_b_empty.to_csv(filepath('mapping_fuzz_match_empty.csv'), index=False, header = True)

In [119]:
#fuzzy match as a function
from fuzzywuzzy import fuzz
def fuzzy_match(str1,str2):
    
    fuzz_ratio = fuzz.ratio(str1, str2)
    jaro_ratio = jaro.jaro_winkler_metric(str1, str2)
    jaro_score = jaro_ratio*100
    average_score = (fuzz_ratio+jaro_score)/2
    
    return fuzz_ratio,jaro_score,average_score

In [120]:
# Specify the column to expand or split (e.g., 'B1')
column = 'METASTORE_SYNONYMS'

#this is to crete a table where original columns will be kept
df_b['split_METASTORE_SYNONYMS'] = df_b[column].str.split('|')
df_Cf = df_b.explode('split_METASTORE_SYNONYMS')

In [121]:
file_name = 'mapping_fuzz_match_keepSYNlist_5.csv'
filepath = os.path.join(new_folder,file_name)

df_Cf.to_csv(filepath,index=False)

In [122]:
new_cols = ['fuzz_ratio','jaro_score','AVG_score']
df_Cf[new_cols] = df_Cf.apply(lambda row: pd.Series(fuzzy_match(row['EFO_Name_split'],row['split_METASTORE_SYNONYMS'])),axis=1)
        

In [123]:
file_name = 'mapping_fuzz_match_score.csv'
filepath = os.path.join(new_folder,file_name)
df_Cf.to_csv(filepath,index=False, header=True)

##### Filter the df_Cf by highest value in either AVG,jaro, or fuzz, and create a new table w/o duplicated column A; which will be reviewed by the SME. 

In [124]:
#create the max_value collumn in df_Cf
df_Cf['max_value']=df_Cf[['fuzz_ratio','jaro_score','AVG_score']].max(axis=1)
#sort the df_Cf by column 'METASTORE_ID' then by max_value in descending order
def sort_df(df):
    df_sorted = df.sort_values(by=['METASTORE_ID','max_value'], ascending=[True,False])
    return df_sorted

df_Cf_sorted = sort_df(df_Cf)

#drop duplicates by 'METASTORE_ID'
def dedup_df(df):
    df_dedup = df.drop_duplicates(subset='METASTORE_ID',keep='first')
    return df_dedup

df_Cf_dedup = dedup_df(df_Cf_sorted)

#sort by the max value for the SME review

df_Cf_dedup_sorted = df_Cf_dedup.sort_values(by=['max_value','METASTORE_ID'], ascending=[False,True])

file_name = 'mapping_fuzz_match_score_highest_SME.csv'
filepath = os.path.join(new_folder,file_name)

df_Cf_dedup_sorted.to_csv(filepath,index=False, header=True)


##### Improvement of this code: 
  Change all the strings into lower case before comparison.
  This code can be paired with the string_